# Rickbot ADK Experiments

For experimentation with the Rickbot Agent.

## Setup

To run this notebook, ensure you've authenticated to Google Cloud and installed the project's dependencies.

To setup the Google Cloud environment:

```bash
source scripts/setup-env.sh
```

Then to install the package dependencies into the virtual environment, use the `uv` tool:

1. From your agent's root directory, run `make install` to set up the virtual environment (`.venv`).
2. In this Jupyter notebook, select the kernel from the `.venv` folder to ensure all dependencies are available.

### Import Libraries

In [ ]:
import asyncio
import json
import mimetypes
import os

import requests
import vertexai
from dotenv import load_dotenv
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.auth import default
from google.genai.types import Blob, Content, Part
from IPython.display import Image, Markdown, display


### Load environment variables

In [ ]:
dotenv_path = os.path.abspath('../.env')

if os.path.exists(dotenv_path):
    print(f"Loading environment variables from: {dotenv_path}")
    load_dotenv(dotenv_path=dotenv_path)
else:
    print(f"Warning: .env file not found at {dotenv_path}")

staging_project_id = os.getenv("GOOGLE_CLOUD_STAGING_PROJECT")
if staging_project_id:
    os.environ["GOOGLE_CLOUD_PROJECT"] = staging_project_id
    print(f"Set GOOGLE_CLOUD_PROJECT environment variable to: {staging_project_id}")    

### Ensure We're Authenticating to the Right Project

In [ ]:
credentials, project_id = default()  # To use ADC
vertexai.init(project="rickbot-adk-dev", location="europe-west4", credentials=credentials)

## Local Testing

In [ ]:
from rickbot_agent.agent import get_agent
from rickbot_agent.personality import get_personalities

In [ ]:
for personality_name, personality in get_personalities().items():
    print(f"{personality_name}: {personality!r}")

In [ ]:
# Helper functions
async def run_conversation(
    runner: Runner,
    query: str,
    user_id: str,
    session_id: str,
    file_path: str | None = None,
):
    """Runs the agent with a sample query and personality."""

    # Construct message
    parts = [Part.from_text(text=query)]
    if file_path:
        mime_type = mimetypes.guess_type(file_path)[0]
        if mime_type:
            with open(file_path, "rb") as f:
                parts.append(Part(inline_data=Blob(data=f.read(), mime_type=mime_type)))

    new_message = Content(role="user", parts=parts)

    print(f"\nSending message with personality '{runner.agent.name}' and session '{session_id}'...")
    final_message = ""
    async for event in runner.run_async(
        user_id=user_id,
        session_id=session_id,
        new_message=new_message,
    ):
        # Check for tool calls - and update "Thinking" header accordingly
        if function_calls := event.get_function_calls():
            tool_name = function_calls[0].name
            display(Markdown(f"_Using tool {tool_name}..._"))
        # Check for agent transfers
        elif event.actions and event.actions.transfer_to_agent:
            agent_name = event.actions.transfer_to_agent
            display(Markdown(f"_Delegating to agent: {agent_name}..._"))
        elif event.is_final_response() and event.content and event.content.parts:
            final_message = event.content.parts[0].text
            display(Markdown("### Final message:"))
            display(Markdown(final_message))

    return session_id, final_message

async def setup_agent(agent_name: str, session_id: str, user_id: str):
    agent = get_agent(agent_name)
    session_service = InMemorySessionService()
    session_id = "test_rick_session"
    user_id = "test_rick_user"

    await session_service.create_session(app_name="rickbot_test", user_id=user_id, session_id=session_id)
    runner = Runner(
        agent=agent, 
        app_name="rickbot_test", 
        session_service=session_service)

    return runner, user_id, session_id

In [ ]:
runner, user_id, session_id = await setup_agent("Rick", "test_rick_session", "test_rick_user")
session_id, final_response = await run_conversation(
    runner=runner,
    query="What's the weather in London today?",
    user_id=user_id,
    session_id = session_id,
)

In [ ]:
# Let's show an image to Yoda
image_path = "../media/rickbot-trans.png"
print(f"Displaying image from {image_path}")
display(Image(image_path))

runner, user_id, session_id = await setup_agent("Yoda", "test_yoda_session", "test_yoda_user")
session_id, final_response = await run_conversation(
    runner=runner,
    query="I've sent you a picture. What do you see?",
    file_path=image_path,
    user_id=user_id,
    session_id = session_id,
)

## If you are using Cloud Run

#### Remote Testing

For more information about authenticating HTTPS requests to Cloud Run services, see:
[Cloud Run Authentication Documentation](https://cloud.google.com/run/docs/triggering/https-request)

Remote testing involves using a deployed service URL instead of localhost.

Authentication is handled using GCP identity tokens instead of local credentials.

Note: the code below does not yet work with IAP enabled.

In [ ]:
ID_TOKEN = get_ipython().getoutput("gcloud auth print-identity-token -q")[0]

In [ ]:
# SERVICE_URL = "YOUR_SERVICE_URL_HERE"  # Replace with your Cloud Run service URL
SERVICE_URL = "https://rickbot-adk-147304899500.europe-west4.run.app"

You'll need to first create a Session

In [ ]:
APP_NAME = "adk_sample_app"
user_id = "test_user_123"
session_data = {"state": {"preferred_language": "English", "visit_count": 1}}

session_url = f"{SERVICE_URL}/apps/{APP_NAME}/users/{user_id}/sessions"
print(f"{session_url=}")
headers = {"Content-Type": "application/json", "Authorization": f"Bearer {ID_TOKEN}"}

session_response = requests.post(session_url, headers=headers, json=session_data)
print(f"Session creation status code: {session_response.status_code}")
print(f"Session creation response: {session_response.json()}")
session_id = session_response.json()["id"]

Then you will be able to send a message

In [ ]:
message_data = {
    "app_name": APP_NAME,
    "user_id": user_id,
    "session_id": session_id,
    "new_message": {"role": "user", "parts": [{"text": "Hello!"}]},
    "streaming": True,
}

message_url = f"{SERVICE_URL}/run_sse"
message_response = requests.post(message_url, headers=headers, json=message_data, stream=True)

print(f"Message send status code: {message_response.status_code}")

# Print streamed response
for line in message_response.iter_lines():
    if line:
        line_str = line.decode("utf-8")
        if line_str.startswith("data: "):
            event_json = line_str[6:]
            event = json.loads(event_json)
            print(f"Received event: {event}")
            content = event.get("content")
            if content and "parts" in content:
                parts = content.get("parts")
                if parts and isinstance(parts, list) and len(parts) > 0:
                    # By the end of the loop, this will hold the text from the last event.
                    final_message = parts[0].get("text", "")

display(Markdown(final_message))